In [26]:
import numpy as np

In [33]:
class Node:
    def __new__(cls, num):
        self = super().__new__(cls)
        
        self.to = set()
        self.fromm = set()
        self.num = num
        return self
        
    def __getnewargs__(self):
        # Return the arguments that *must* be passed to __new__
        return (self.num,)
        
    def __hash__(self):
        return self.num
    
    def __eq__(self, node):
        return self.num == node.num
        
    def connectTo(self, node):
        self.to.add(node)
        node.fromm.add(self)

In [34]:
class Graph:
    def __init__(self, size=0):
        self.nodesByNum = dict()
        for i in range(size):
            self.nodesByNum[i] = Node(i)
        self.nodes = list(self.nodesByNum.values())

    """Возвращает конденсацию графа"""
    def condence(self):
        for node in self.nodes:
            node.visited = False
        out_order = []
        for node in self.nodes:
            if not node.visited:
                self.reverseDfs(node, out_order)
        
        new_graph = Graph()
        for node in self.nodes:
            node.visited = False
        
        newNodesByOldNums = dict()
        for node in reversed(out_order):
            if not node.visited:
                visited_nodes = []
                self.dfs(node, visited_nodes)
                new_node = Node(len(new_graph.nodes))
                new_graph.nodes.append(new_node)
                for node2 in visited_nodes:
                    newNodesByOldNums[node2.num] = new_node
                                
        for node in self.nodes:
            for node2 in node.to:
                newNodesByOldNums[node.num].connectTo(newNodesByOldNums[node2.num])
        
        for i in range(len(self.nodesByNum)):
            new_graph.nodesByNum[i] = newNodesByOldNums[self.nodesByNum[i].num]
            
        return new_graph  
         
    def dfs(self, node, visited_nodes):
        node.visited = True
        for node2 in node.to:
            if not node2.visited:
                self.dfs(node2, visited_nodes)
        visited_nodes.append(node)
    
    def reverseDfs(self, node, out_order):
        node.visited = True
        for node2 in node.fromm:
            if not node2.visited:
                self.reverseDfs(node2, out_order)
        out_order.append(node)
    
    def setAdjMatrix(self, matrix):
        for x in range(matrix.shape[0]):
            for y in range(matrix.shape[1]):
                if matrix[x,y] == 1:
                    self.nodes[x].connectTo(self.nodes[y])
    
    def getAdjMatrix(self):
        matrix = np.zeros((len(self.nodes), len(self.nodes)), dtype=np.int32)
        for x in range(len(self.nodes)):
            for node in self.nodes[x].to:
                matrix[x, node.num] = 1
        return matrix
    
    def hasEdgeByNums(self, num1, num2):
        n1 = self.nodesByNum[num1]
        n2 = self.nodesByNum[num2]
        if n1 == n2:
            return True
        for n in n1.to:
            if n == n2:
                return True
        return False
    
    def addEdgeByNums(self, num1, num2):
        self.nodesByNum[num1].connectTo(self.nodesByNum[num2])

In [1]:
import copy

def usefulOperation(graph, operation):
    for pair in operation:
        if not graph.hasEdgeByNums(pair[0], pair[1]):
            return True
    return False

def applyOperation(graph, operation):
    for pair in operation:
        graph.addEdgeByNums(pair[0], pair[1])

def isTrapped(graph, operations, condensed=False):
    if not condensed:
        graph = graph.condence()
        node0 = graph.nodesByNum[0]
        for node in node0.to:
            if node != node0:
                return True
        operations = list(filter(lambda op: usefulOperation(graph, op), operations))
    if len(operations) == 0:
        return False
    toApply = operations[-1]
    if isTrapped(graph, operations[:-1], True):
        return True
    graph_copy = copy.deepcopy(graph)
    applyOperation(graph_copy, toApply)
    return isTrapped(graph_copy, operations[:-1], False)